## URBAN EXTENT

In [ ]:
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed

temp_dir = os.path.join(os.path.abspath("./"), 'temp')
os.makedirs(temp_dir, exist_ok=True)

json_pipe_base = """ { "pipeline": [ %s ] } """


"""
Scalar: Scaling value
    Increase this parameter for terrains with lots of height variation.
Slope: slope parameter
    Which is a measure of “slope tolerance”. Increase this parameter for terrains with lots of height variation.
    Should be set to something higher than 0.1 and not higher than 1.2.
Threshold:  Elevation threshold
    Set this parameter to the minimum height (in meters) that you expect non-ground objects to be.
Window: Window radius parameter (in meters) 
    Corresponds to the size of the largest feature (building, trees, etc.) to be removed. Should be set to a value higher than 10.
    
Threshold option has the biggest impact on results (per OpenDroneMap docs)
"""


filterCrop = """
    {
        "type":"filters.crop",
        "bounds":"([986710, 989030],[503890, 505700])"
    }"""
filterSMRF = """
    {
        "type": "filters.smrf",
        "cell": %s,
        "dir": "%s",
        "returns":"first,last,intermediate,only",
        "scalar": 1.0,
        "slope": 0.3,
        "threshold": 1,
        "window": %s     
    }"""
filterPMF = """
    {
        "type": "filters.pmf",
        "cell_size": %s,
        "initial_distance": %s,
        "returns":"first,last,intermediate,only",
        "max_distance": %s,
        "max_window_size": %s,
        
    }"""
filterGround = """
    {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    }"""
writeLAZ = """
    {
        "type":"writers.las",
        "compression":"laszip",
        "filename":"%s"
    }"""
writeTIFF = """
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW"
    }"""


laz_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full"
dtm_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/DTM_2ft"
os.makedirs(dtm_folder, exist_ok=True)


infiles = []
for f in os.listdir(laz_folder):
    infile = os.path.join(laz_folder, f)
    outfile = os.path.join(dtm_folder, f.replace(".laz", "_DTM.tif"))
    if f.endswith(".laz"):# and not os.path.exists(outfile):
        files = {}
        files["in_file"] = infile
        files["out_file"] = outfile
        infiles.append(files)
        
def createDTM(file_dict):
    infile = file_dict["in_file"]
    ground_tiff = file_dict["out_file"]

    ground_laz = "../EPCExtent_30cm/Elevation_80cmNPS/RGB/ground/" + os.path.basename(infile)
    
    filterSMRF = """
    {
        "type": "filters.smrf",
        "cell": %s,
        "dir": "%s",
        "returns":"first,last,intermediate,only",
        "scalar": 1.0,
        "slope": 0.18,
        "threshold": %s,
        "window": %s     
    }"""
    #filterPMF = """
    #{
    #    "type": "filters.pmf",
    #    "cell_size": %s,
    #    "initial_distance": %s,
    #    "returns":"first,last,intermediate,only",
    #    "max_distance": %s,
    #    "max_window_size": %s,
    #    "slope": %s
    #}"""
    
    #res = 0.984251969
    res = 2 # match 2015 Ortho
    
    win_size = 30
    
    if os.path.exists(ground_laz):
        pipeline_json = """ "%s" """ % ground_laz
    else:
        pipeline_json = """ "%s" """ % infile
        #pipeline_json += ',' + filterCrop
        pipeline_json += ',' + filterSMRF % (25, temp_dir.replace("\\", "/"), 1.5, win_size)
        #pipeline_json += ',' + filterPMF % (8, 0.15, 40, 50, 1.5)
        pipeline_json += ',' + filterGround
        pipeline_json += ',' + writeLAZ % (ground_laz)
        
    if not os.path.exists(ground_tiff):
        pipeline_json += ',' + writeTIFF % (ground_tiff, res, res * 10)
        
    pipeline_json = pipeline_json.replace("\\","/")
    
    pipeline_json = json_pipe_base % pipeline_json
    #print(pipeline_json)
    
    pipeline = pdal.Pipeline(pipeline_json)
    pipeline.loglevel = 8

    start = datetime.now()
    pipeline.execute()
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {os.path.basename(outfile)} - {end-start} elapsed")
    
    return pipeline_json
    

print("\nBeginning DTM Generations for {} QQuads\n".format(len(infiles)))

for f in infiles:
    #if "E0980_N510_1" in f['out_file']:
    print(f"Starting on {f['out_file']} at {datetime.now()}")
    json_pipe = createDTM(f)
    
print("\n\n...FINISHED\n\n")